In [ ]:
### 1.Preprocessing data 

In [ ]:
### 2.CustomDataset


In [ ]:
### 3.DATALOADER

In [ ]:
### 4.Model 

import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size // 2, num_layers=num_layers, 
                            batch_first=True, dropout=dropout, bidirectional=True)

    def forward(self, x):
        # x shape: (batch_size, seq_len, n_features)
        outputs, (h, c) = self.lstm(x)
        # Concatenate the hidden states for both directions
        h = torch.cat((h[-2], h[-1]), dim=1)
        return outputs, h

class Decoder(nn.Module):
    def __init__(self, hidden_size, max_label_length, num_classes, num_layers, dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.max_label_length = max_label_length
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=num_layers, 
                            batch_first=True, dropout=dropout)
        self.out_layer = nn.Linear(hidden_size, num_classes)

    def forward(self, h, encoder_output):
        # Prepare initial states
        h = h.unsqueeze(0).repeat(self.num_layers, 1, 1)  # Repeat for each layer
        c = torch.zeros_like(h)  # Initial cell states
        # Prepare decoder inputs, initial hidden state only, inputs will be generated
        inputs = torch.zeros(encoder_output.size(0), self.max_label_length, self.hidden_size).to(h.device)
        outputs, (hn, cn) = self.lstm(inputs, (h, c))
        outputs = self.out_layer(outputs)
        outputs = outputs.transpose(1, 2)

        return outputs

class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x):
        encoder_output, last_hidden_state = self.encoder(x)
        decoder_output = self.decoder(last_hidden_state, encoder_output)
        return decoder_output



# Define the dimensions
hidden_size = 1024
batch_size = 32
num_substrokes = 60
substroke_length = 54
num_classes =33
max_label_length = 1
dropout= 0.2
num_layer= 3


encoder = Encoder(input_size=substroke_length, hidden_size=hidden_size, num_layers=num_layer, dropout=dropout)
decoder = Decoder(hidden_size=hidden_size, max_label_length=max_label_length, num_classes=num_classes, num_layers=num_layer, dropout=dropout)
model = EncoderDecoder(encoder, decoder)

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

model=model.to(device)

print("Model architecture:")
print(model, '\n')

# Test the updated model
input_tensor = torch.randn(batch_size, num_substrokes, substroke_length).to(device)
output = model(input_tensor)
print("Output shape:", output.shape)  

# Expected Shape: (batch_size,num_classes, max_label_length)

In [ ]:
### 5.Trainig 

In [ ]:
### 6.Evaluation 